<a href="https://colab.research.google.com/github/benayas1/ALV_Framework/blob/master/baseline_4capas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
import warnings
warnings.filterwarnings('ignore')

#the basics
import pandas as pd, numpy as np
import math, json, gc, random, os, sys
from matplotlib import pyplot as plt
from tqdm import tqdm

#tensorflow deep learning basics
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L

import time

#for model evaluation
from sklearn.model_selection import train_test_split, KFold

ROOT = '/content/drive/My Drive/Kaggle/covid/'
PATH = 'baseline_4layers/'

In [3]:
# Seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
SEED=42
seed_everything(SEED)

def adjust(public_df, private_df, public_pred, private_pred):
    predictions = []

    for df, preds in [(public_df, public_pred), (private_df, private_pred)]:
        for i, uid in enumerate(df.id):
            single_pred = preds[i]

            single_df = pd.DataFrame(single_pred, columns=target_cols)
            single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

            predictions.append(single_df)

    preds_df = pd.concat(predictions)
    return preds_df
    
def blend_column(dfs, weights, column):
    values = np.array([df[column]*w for df, w in list(zip(dfs, weights))])
    values = np.sum(values, axis=0)
    return values

def get_fold(fold, folds):
    train_idx = np.concatenate([folds[i] for i in range(len(folds)) if i != fold])
    val_idx = folds[fold]
    return train_idx, val_idx

def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type'], map_tokens=None):
    return np.transpose(
        np.array(
            df[cols]
            .applymap(lambda seq: [map_tokens[x] for x in seq])
            .values
            .tolist()
        ),
        (0, 2, 1)
    )

def load_predict(model, path, public_inputs, private_inputs):
    short = build_model(model, seq_len=107, pred_len=107)
    short.load_weights(path)
    long =  build_model(model, seq_len=130, pred_len=130)
    long.load_weights(path)
    public = short.predict(public_inputs)
    private = long.predict(private_inputs)
    return public, private

def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

def pairs(seq, structure):
    pairs = seq.copy()
    stack = []
    for i,s in enumerate(structure):
        if s == '(':
            stack.append((i,seq[i]))
        if s == ')':
            pos, pair = stack.pop()
            pairs[pos] = seq[i]
            pairs[i] = pair
    return pairs

In [4]:
#get comp data
train = pd.read_json(ROOT+'train.json', lines=True)
test = pd.read_json(ROOT+'test.json', lines=True)
sample_sub = pd.read_csv(ROOT+'sample_submission.csv')

# Brief EDA

In [5]:
train['pairs'] = train.apply(lambda x: pairs(list(x['sequence']), list(x['structure'])), axis=1)
test['pairs'] = test.apply(lambda x: pairs(list(x['sequence']), list(x['structure'])), axis=1)
print(train.shape)
if ~ train.isnull().values.any(): print('No missing values')
print(test.shape)
if ~ test.isnull().values.any(): print('No missing values')
print(sample_sub.shape)
if ~ sample_sub.isnull().values.any(): print('No missing values')

(2400, 20)
No missing values
(3634, 8)
No missing values
(457953, 6)
No missing values


# Processing

**From the data [description tab](https://www.kaggle.com/c/stanford-covid-vaccine/data), we must predict multiple ground truths in this competition, 5 to be exact. While the submission requires all 5, only 3 are scored: `reactivity`, `deg_Mg_pH10` and `deg_Mg_50C`**

**The training features we are given are as follows:**

* **id** - An arbitrary identifier for each sample.
* **seq_scored** - (68 in Train and Public Test, 91 in Private Test) Integer value denoting the number of positions used in scoring with predicted values. This should match the length of `reactivity`, `deg_*` and `*_error_*` columns. Note that molecules used for the Private Test will be longer than those in the Train and Public Test data, so the size of this vector will be different.
* **seq_length** - (107 in Train and Public Test, 130 in Private Test) Integer values, denotes the length of `sequence`. Note that molecules used for the Private Test will be longer than those in the Train and Public Test data, so the size of this vector will be different.
* **sequence** - (1x107 string in Train and Public Test, 130 in Private Test) Describes the RNA sequence, a combination of `A`, `G`, `U`, and `C` for each sample. Should be 107 characters long, and the first 68 bases should correspond to the 68 positions specified in `seq_scored` (note: indexed starting at 0).
* **structure** - (1x107 string in Train and Public Test, 130 in Private Test) An array of `(`, `)`, and `.` characters that describe whether a base is estimated to be paired or unpaired. Paired bases are denoted by opening and closing parentheses e.g. (....) means that base 0 is paired to base 5, and bases 1-4 are unpaired.
* **reactivity** - (1x68 vector in Train and Public Test, 1x91 in Private Test) An array of floating point numbers, should have the same length as `seq_scored`. These numbers are reactivity values for the first 68 bases as denoted in `sequence`, and used to determine the likely secondary structure of the RNA sample.
* **deg_pH10** - (1x68 vector in Train and Public Test, 1x91 in Private Test) An array of floating point numbers, should have the same length as `seq_scored`. These numbers are reactivity values for the first 68 bases as denoted in `sequence`, and used to determine the likelihood of degradation at the base/linkage after incubating without magnesium at high pH (pH 10).
* **deg_Mg_pH10** - (1x68 vector in Train and Public Test, 1x91 in Private Test) An array of floating point numbers, should have the same length as `seq_scored`. These numbers are reactivity values for the first 68 bases as denoted in `sequence`, and used to determine the likelihood of degradation at the base/linkage after incubating with magnesium in high pH (pH 10).
* **deg_50C** - (1x68 vector in Train and Public Test, 1x91 in Private Test) An array of floating point numbers, should have the same length as `seq_scored`. These numbers are reactivity values for the first 68 bases as denoted in `sequence`, and used to determine the likelihood of degradation at the base/linkage after incubating without magnesium at high temperature (50 degrees Celsius).
* **deg_Mg_50C** - (1x68 vector in Train and Public Test, 1x91 in Private Test) An array of floating point numbers, should have the same length as `seq_scored`. These numbers are reactivity values for the first 68 bases as denoted in `sequence`, and used to determine the likelihood of degradation at the base/linkage after incubating with magnesium at high temperature (50 degrees Celsius).
* **`*_error_*`** - An array of floating point numbers, should have the same length as the corresponding `reactivity` or `deg_*` columns, calculated errors in experimental values obtained in `reactivity` and `deg_*` columns.
* **predicted_loop_type** - (1x107 string) Describes the structural context (also referred to as 'loop type')of each character in `sequence`. Loop types assigned by bpRNA from Vienna RNAfold 2 structure. From the bpRNA_documentation: S: paired "Stem" M: Multiloop I: Internal loop B: Bulge H: Hairpin loop E: dangling End X: eXternal loop

In [6]:
#target columns
target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

**Now we create a dictionary to help us map `sequence`, `structure`, and `predicted_loop_type` to columns we can feed a model:**

In [7]:
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

In [8]:
train = train[train.signal_to_noise > 1]  # some signals have a lot of noise
train_inputs = preprocess_inputs(train, cols=['sequence', 'structure', 'predicted_loop_type', 'pairs'], map_tokens=token2int)
train_labels = np.array(train[target_cols].values.tolist()).transpose((0, 2, 1))
print('Train Input shape', train_inputs.shape)

Train Input shape (2096, 107, 4)


# Model

**We begin with a simple GRU model taken from the one and only [Xhlulu](https://www.kaggle.com/xhlulu)'s notebook [here](https://www.kaggle.com/xhlulu/openvaccine-simple-gru-model)**

**From the documentation of this competition, you can read that due to technical reasons, measurements cannot be carried out on the final bases of the RNA sequences we have just have experimental data (as ground truths) in 5 conditions for the first 68 bases. This means we must truncate the output of our model:**

In [15]:
def gru_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
                                tf.keras.layers.GRU(hidden_dim,
                                dropout=dropout,
                                return_sequences=True,
                                kernel_initializer = 'orthogonal'))

def lstm_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
                                tf.keras.layers.LSTM(hidden_dim,
                                dropout=dropout,
                                return_sequences=True,
                                kernel_initializer = 'orthogonal'))

def build_model(model,seq_len=107, pred_len=68, dropout=0.5,
                embed_dim=75, hidden_dim=192, n_sequences=4):
    
    inputs = tf.keras.layers.Input(shape=(seq_len, n_sequences))

    embed = tf.keras.layers.Embedding(input_dim=len(token2int), output_dim=embed_dim)(inputs)
    reshaped = tf.reshape(embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))
    
    reshaped = tf.keras.layers.SpatialDropout1D(.2)(reshaped)
    
    if model == 0:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        
    if model == 1:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        
    if model == 2:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        
    if model == 3:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
    
    #only making predictions on the first part of each sequence
    truncated = hidden[:, :pred_len]
    
    out = tf.keras.layers.Dense(5, activation='linear')(truncated)

    model = tf.keras.Model(inputs=inputs, outputs=out)

    #some optimizers
    adam = tf.optimizers.Adam()
    radam = tfa.optimizers.RectifiedAdam()
    lookahead = tfa.optimizers.Lookahead(adam, sync_period=6)
    ranger = tfa.optimizers.Lookahead(radam, sync_period=6)
    
    model.compile(optimizer = adam, loss=MCRMSE)
    
    return model

# Training

**Create train/val split now so both models are trained and evaluated on the same samples:**

In [16]:
cv = KFold(5, shuffle=True, random_state=SEED)
folds = {}
for i, (train_index, val_index) in enumerate(cv.split(train_inputs)):
    folds[i] = val_index
    
def get_fold(fold, folds):
    train_idx = np.concatenate([folds[i] for i in range(len(folds)) if i != fold])
    val_idx = folds[fold]
    return train_idx, val_idx

In [12]:
#train_inputs, val_inputs, train_labels, val_labels = train_test_split(train_inputs, train_labels, test_size=.01, random_state=34)

**We will use a simple learning rate callback for now:**

In [17]:
lr_callback = tf.keras.callbacks.ReduceLROnPlateau()
#lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=2, mode='min', min_delta=0.00001, cooldown=0, min_lr=1e-8)
#lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, verbose=2, mode='min', min_delta=0.001, cooldown=0, min_lr=1e-8)

In [18]:
def fit(model_data, X, y, epochs=150, batch_size=64, verbose=0, patience=13, folds=None):
    
    paths = []
    errors = []
    for i in range(len(folds)):
        start = time.time()
        print('Training fold',i)
        train_idx, val_idx = get_fold(i, folds)
        
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[val_idx], y[val_idx]
        
        gru = build_model(model=model_data[0], n_sequences=X_train.shape[2])
        path = model_data[1]+'_fold_'+str(i)+'.h5'
        sv_gru = tf.keras.callbacks.ModelCheckpoint(path)
        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=patience)

        history = gru.fit(
            X_train, y_train, 
            validation_data=(X_test, y_test),
            batch_size=batch_size,
            epochs=epochs,
            callbacks=[lr_callback, sv_gru, es],
            verbose = verbose
        )
        
        best_epoch = np.argmin(history.history['val_loss'])
        min_val_loss = history.history['val_loss'][best_epoch]
        min_train_loss = history.history['loss'][best_epoch]
        paths.append(path)
        errors.append(min_val_loss)

        print(f"\tMin training loss={min_train_loss}, min validation loss={min_val_loss}, elapsed {time.time()-start}")
        
    print(f"\tOOF CV loss={np.mean(errors)}")
    return list(zip([model_data[0]]*len(paths) , paths)), errors

# Training

In [19]:
model_results = []
errors = []
model_data = [(0,PATH+'model0'), (1,PATH+'model1'), (2,PATH+'model2'), (3,PATH+'model3')]

### 1. GRU

In [20]:
res, error = fit(model_data[0], train_inputs, train_labels, batch_size=16, verbose=2, patience=10, folds=folds)
model_results += res
errors += error

Training fold 0
Epoch 1/150
105/105 - 7s - loss: 0.4086 - val_loss: 0.3606
Epoch 2/150
105/105 - 5s - loss: 0.3585 - val_loss: 0.3548
Epoch 3/150
105/105 - 5s - loss: 0.3408 - val_loss: 0.3329
Epoch 4/150
105/105 - 5s - loss: 0.3290 - val_loss: 0.3183
Epoch 5/150
105/105 - 5s - loss: 0.3162 - val_loss: 0.3072
Epoch 6/150
105/105 - 6s - loss: 0.3044 - val_loss: 0.2913
Epoch 7/150
105/105 - 5s - loss: 0.2944 - val_loss: 0.2848
Epoch 8/150
105/105 - 6s - loss: 0.2858 - val_loss: 0.2721
Epoch 9/150
105/105 - 5s - loss: 0.2767 - val_loss: 0.2728
Epoch 10/150
105/105 - 5s - loss: 0.2691 - val_loss: 0.2568
Epoch 11/150
105/105 - 6s - loss: 0.2631 - val_loss: 0.2510
Epoch 12/150
105/105 - 5s - loss: 0.2565 - val_loss: 0.2459
Epoch 13/150
105/105 - 5s - loss: 0.2527 - val_loss: 0.2456
Epoch 14/150
105/105 - 6s - loss: 0.2498 - val_loss: 0.2418
Epoch 15/150
105/105 - 5s - loss: 0.2449 - val_loss: 0.2398
Epoch 16/150
105/105 - 5s - loss: 0.2427 - val_loss: 0.2399
Epoch 17/150
105/105 - 5s - loss:

### 2. LSTM

In [21]:
res, error = fit(model_data[1], train_inputs, train_labels, batch_size=16,verbose=0, patience=10, folds=folds)
model_results += res
errors += error

Training fold 0
	Min training loss=0.16203252971172333, min validation loss=0.21419930458068848, elapsed 420.7448251247406
Training fold 1
	Min training loss=0.17029674351215363, min validation loss=0.20865190029144287, elapsed 379.589262008667
Training fold 2
	Min training loss=0.17418727278709412, min validation loss=0.2194335162639618, elapsed 334.0083656311035
Training fold 3
	Min training loss=0.15271352231502533, min validation loss=0.2181905210018158, elapsed 489.6445505619049
Training fold 4
	Min training loss=0.15969733893871307, min validation loss=0.21242062747478485, elapsed 447.50039052963257
	OOF CV loss=0.21457917392253875


### 3. LSTM + GRU

In [22]:
res, error = fit(model_data[2], train_inputs, train_labels, batch_size=16,verbose=0, folds=folds)
model_results += res
errors += error

Training fold 0
	Min training loss=0.15079553425312042, min validation loss=0.2133994847536087, elapsed 685.0200002193451
Training fold 1
	Min training loss=0.16434916853904724, min validation loss=0.20793390274047852, elapsed 483.7919330596924
Training fold 2
	Min training loss=0.15954819321632385, min validation loss=0.214729443192482, elapsed 662.0508465766907
Training fold 3
	Min training loss=0.1562618464231491, min validation loss=0.2181054949760437, elapsed 633.3748867511749
Training fold 4
	Min training loss=0.15845908224582672, min validation loss=0.21068955957889557, elapsed 714.9008951187134
	OOF CV loss=0.2129715770483017


### 4. GRU + LSTM

In [23]:
res, error = fit(model_data[3], train_inputs, train_labels, batch_size=16,verbose=0, folds=folds)
model_results += res
errors += error

Training fold 0
	Min training loss=0.15797114372253418, min validation loss=0.21113689243793488, elapsed 493.18807888031006
Training fold 1
	Min training loss=0.15805599093437195, min validation loss=0.20538043975830078, elapsed 488.47091031074524
Training fold 2
	Min training loss=0.15113264322280884, min validation loss=0.21359628438949585, elapsed 662.4131855964661
Training fold 3
	Min training loss=0.1477593183517456, min validation loss=0.21534012258052826, elapsed 617.0750877857208
Training fold 4
	Min training loss=0.15797513723373413, min validation loss=0.20937970280647278, elapsed 496.25954246520996
	OOF CV loss=0.21096668839454652


In [24]:
print('CV error is ', np.mean(errors))

CV error is  0.21343172416090966


# Inference

In [26]:
public_df = test.query("seq_length == 107").copy()
private_df = test.query("seq_length == 130").copy()

public_inputs = preprocess_inputs(public_df, cols=['sequence', 'structure', 'predicted_loop_type', 'pairs'], map_tokens=token2int)
private_inputs = preprocess_inputs(private_df, cols=['sequence', 'structure', 'predicted_loop_type', 'pairs'], map_tokens=token2int)

**Predict twice, one for the public leaderboard, the other for the private leaderboard:**

In [27]:
def load_trained_models(model, path):
    short = build_model(model, seq_len=107, pred_len=107)
    short.load_weights(path)
    long =  build_model(model, seq_len=130, pred_len=130)
    long.load_weights(path)
    return predict_pair(short, long)
    #return short, long

def predict_pair(short, long):
    public = short.predict(public_inputs)
    private = long.predict(private_inputs)
    return public, private

In [ ]:
def get_n(f):
    if f[6:14]=='gru_lstm': return 3
    if f[6:14]=='lstm_gru': return 2
    if f[6:10]=='lstm': return 1
    if f[6:9]=='gru': return 0

In [ ]:
#from glob import glob
#model_results = glob('*.h5')
#model_results = [( get_n(f), f) for f in model_results]
#model_results = [(n, f) for n,f in model_results if n==3]
#model_results

In [28]:
#build all models
#models = [load_trained_models(m,p) for m,p in tqdm(model_results) ]

#and predict
#predictions = [predict_pair(short, long) for short, long in tqdm(models)]
predictions = [load_trained_models(m,p) for m,p in tqdm(model_results) ]



100%|██████████| 20/20 [03:49<00:00, 11.46s/it]


# Postprocessing

### Adjust Output
According to pubic and private test dataset

In [29]:
dfs  = [adjust(public_df, private_df, public_preds, private_preds) for public_preds, private_preds in tqdm(predictions)]
weights = [1/len(dfs)] * len(dfs)

100%|██████████| 20/20 [01:10<00:00,  3.55s/it]


### Blending

In [30]:
blend_preds_df = pd.DataFrame({'id_seqpos':dfs[0]['id_seqpos']})
for c in target_cols:
    blend_preds_df[c] = blend_column(dfs, weights, c)

In [31]:
submission = sample_sub[['id_seqpos']].merge(blend_preds_df, on=['id_seqpos'])

#sanity check
submission.head()

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.746345,0.649274,1.960138,0.547165,0.765299
1,id_00073f8be_1,2.223649,3.185862,3.910727,3.246317,2.777526
2,id_00073f8be_2,1.573120,0.615004,0.661253,0.725471,0.678785
3,id_00073f8be_3,1.317544,1.148795,1.186439,1.658210,1.869232
4,id_00073f8be_4,0.821595,0.597514,0.543067,0.859756,0.913399


In [35]:
submission.to_csv(ROOT+PATH+'submission.csv', index=False)
print('Submission saved')

Submission saved
